<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 3: Aproximación de autovalores

In [1]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [2]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np

from scipy.linalg import orth

In [3]:
from anmi.genericas import norma, print_verbose

from anmi.T2 import factorizacion_QR
from anmi.T3 import matriz_krylov, sucesion_krylov, potencia_iterada, metodo_autovals_QR

### Sucesiones de Krylov
Sea $A$ una matriz (aplicación lineal) y $x$ un vector. Si aplicamos la multiplicación de $A$ por $x$ de manera iterada obtenemos una serie de vectores $\{x, Ax, A^2x, A^3x, \cdots\}$. Si $x$ no es un autovector de $A$, entonces esa sucesión tendrá $n$ (dimensión de $A$) vectores independientes. Si $x$ es un autovector, con su autovalor $\lambda$, entonces la sucesión de vectores será, $\{x, \lambda x, \lambda^2x, \lambda^3x, \cdots\}$. Estas sucesiones de vectores se llaman *sucesiones de Krylov*.

Por otra parte, por el teorema de Cayley-Hamilton se tiene que $A^nx$ tiene que ser una combinación lineal de los siguientes elementos de la sucesión, es decir:
$$(-1)^nA^n + a_{n-1}A^{n-1} + \cdots + a_1A + a_0I = 0$$

Luego si tomamos $a = \begin{bmatrix}a_0\\a_1\\ \cdots \\ a_n\end{bmatrix}$ se tiene que
$$(x|Ax|\cdots|A^{n-1}x)a = (-1)^{n+1}A^nx$$

Y si resolvemos $a$, entonces se tienen los coeficientes del polinómio característico $p(\lambda) = a_0 + a_1\lambda + a_2\lambda^2 + \cdots + a_n\lambda^n$$


In [4]:
help(matriz_krylov)

Help on function matriz_krylov in module anmi.T3:

matriz_krylov(A, x, n_iters=None)
    Genera una matriz de krylov dada una matriz A y un vector x. Cada columna de la matriz es la iteración i de A^i*x.
    
    Args:
        A (matriz): Matriz de aplicación
        x (vector): Vector base
        n_iters (int, optional): Número de iteraciones. Por defecto es el número de filas de A + 1 (garantiza que la matriz de krylov tiene una combinación lineal).
    
    Returns:
        m_krylov: Matriz con las aplicaciones de krylov por columna.



In [5]:
# EJERCICIO 26
A = mat([[1, 1, 1], [0, 2, 2], [3, -1, 0]])
x = mat([[1, 0, 0]]).T

m_krylov = matriz_krylov(A, x)
m_krylov

Matrix([
[1, 1, 4, 13],
[0, 0, 6, 18],
[0, 3, 3,  6]])

In [6]:
sk, a = sucesion_krylov(A, x)
sk

Poly(-lambda**3 + 3*lambda**2 - lambda + 2, lambda, domain='ZZ')

In [7]:
# EJEMPLO 15
A = mat([[2, -1, 0], [-1, 2, -1], [0, -1, 2]])
x = mat([[-1, 0, 1]]).T

matriz_krylov(A, x)

Matrix([
[-1, -2, -4, -8],
[ 0,  0,  0,  0],
[ 1,  2,  4,  8]])

### Método de la potencia iterada

En el método de la potencia iterada, se aplica la matriz de krylov hasta una potencia determinada, $k$. Entonces, se tiene que 
$$\lim_{k \to \infty} \frac{A^kw}{A^{k-1}w} = |\lambda_1|$$
Es decir, el mayor autovalor.

Además, si tomamos $ B= A^{-1} $, tenemos que
$$\lim_{k \to \infty} \frac{B^kw}{B^{k-1}w} = \frac{1}{|\lambda_n|}$$
Donde $\lambda_n$ es el menor autovalor.

In [8]:
help(potencia_iterada)

Help on function potencia_iterada in module anmi.T3:

potencia_iterada(A, x, n_iters, devolver_ultimo=True)
    Aplica el método de la potencia iterada para calcular el mayor autovalor de la matriz, usando el método de Krylov.
    
    Args:
        A (matriz): Matriz aplicación
        x (vector): Vector base para el método. Si el vector es autovector dará fallo.
        n_iters (int): Número de iteraciones
        devolver_ultimo (bool, optional): Si True, devuelve el vector de la última iteración. Si False, devuelve todas las iteraciones.
    
    Returns:
        m_cocientes (matriz, vector): matriz/vector con el número de filas igual al de A, con los cocientes. Los números deberían tender al mayor autovalor de A.



In [9]:
A.eigenvals()

{2 - sqrt(2): 1, 2: 1, sqrt(2) + 2: 1}

In [10]:
x = mat([[-2, 0, 1]]).T

matriz_krylov(A, x, 17)

Matrix([
[-2, -4, -9, -22, -58, -164, -492, -1544, -5000, -16528, -55344, -186784, -633376, -2153792, -7336128, -25012352, -85328000],
[ 0,  1,  4,  14,  48,  164,  560,  1912,  6528,  22288,  76096,  259808,  887040,  3028544, 10340096,  35303296, 120532992],
[ 1,  2,  3,   2, -10,  -68, -300, -1160, -4232, -14992, -52272, -180640, -621088, -2129216, -7286976, -24914048, -85131392]])

In [11]:
x = mat([[-1, 0, 0]]).T

np.array(potencia_iterada(A, x, 30, devolver_ultimo=False)[:, -3:], dtype=float)

array([[3.41421098, 3.41421205, 3.41421267],
       [3.41421356, 3.41421356, 3.41421356],
       [3.41421615, 3.41421508, 3.41421445]])

In [12]:
x = mat([[-1, 0, 0]]).T

np.array(potencia_iterada(A, x, 300, devolver_ultimo=True), dtype=float)

array([[3.41421356],
       [3.41421356],
       [3.41421356]])

In [13]:
N(2+sqrt(2))  # Vemos que converge al mayor autovalor

3.41421356237309

In [14]:
np.array(potencia_iterada(A**-1, x, 300, devolver_ultimo=True), dtype=float)

array([[1.70710678],
       [1.70710678],
       [1.70710678]])

In [15]:
1/N(2-sqrt(2))  # Y lo mismo con el menor

1.70710678118655

In [16]:
# Si tomamos una matriz ortogonal, el metodo de la potencia no tiene validez 
# porque se requiere que haya autovalores dominantes, y en este caso los 
# autovalores tienen módulo 1.

dict_QR = factorizacion_QR(A)
Q = dict_QR['Q']

Q

Aplicamos QR con Gram Schmidt
Q es 
[[2*sqrt(5)/5 3*sqrt(70)/70 sqrt(14)/14]
 [-sqrt(5)/5 3*sqrt(70)/35 sqrt(14)/7]
 [0 -sqrt(70)/14 3*sqrt(14)/14]]
R es 
[[sqrt(5) -4*sqrt(5)/5 sqrt(5)/5]
 [0 sqrt(70)/5 -8*sqrt(70)/35]
 [0 0 2*sqrt(14)/7]]
D es 
[[sqrt(5) 0 0]
 [0 sqrt(70)/5 0]
 [0 0 2*sqrt(14)/7]]


Matrix([
[2*sqrt(5)/5, 3*sqrt(70)/70,   sqrt(14)/14],
[ -sqrt(5)/5, 3*sqrt(70)/35,    sqrt(14)/7],
[          0,  -sqrt(70)/14, 3*sqrt(14)/14]])

In [17]:
Q * Q.T

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [18]:
Q.eigenvals()

{-1/2 + 3*sqrt(70)/70 + 3*sqrt(14)/28 + sqrt(5)/5 + sqrt(70)*I*sqrt(6*sqrt(14) + 20*sqrt(5) + 73)/140: 1,
 -1/2 + 3*sqrt(70)/70 + 3*sqrt(14)/28 + sqrt(5)/5 - sqrt(70)*I*sqrt(6*sqrt(14) + 20*sqrt(5) + 73)/140: 1,
 1: 1}

In [19]:
N(-1/2 + 3*sqrt(70)/70 + 3*sqrt(14)/28 + sqrt(5)/5 + sqrt(70)*I*sqrt(6*sqrt(14) + 20*sqrt(5) + 73)/140)

0.706674041168913 + 0.707539256534928*I

In [20]:
matriz_krylov(Q, x, 5)
N(matriz_krylov(N(Q), x, 30), 4)

Matrix([
[-1.0, -0.8944, -0.6396, -0.3851, -0.2802, -0.3864, -0.6414, -0.8957,      -1.0,    -0.8932, -0.6379, -0.3839, -0.2802, -0.3876, -0.6432, -0.8969,     -1.0,   -0.8919, -0.6361, -0.3826, -0.2802, -0.3889, -0.6449, -0.8981,     -1.0,   -0.8907, -0.6344, -0.3814, -0.2802, -0.3901],
[   0,  0.4472,  0.7207,    0.66,  0.3008, -0.1463, -0.4189, -0.3571,  0.002789,     0.4497,  0.7214,  0.6586,   0.298, -0.1487, -0.4196, -0.3557, 0.005581,    0.4522,  0.7222,  0.6571,  0.2952, -0.1512, -0.4203, -0.3542, 0.008377,    0.4547,  0.7229,  0.6556,  0.2924, -0.1537],
[   0,       0, -0.2673,  -0.645, -0.9116, -0.9107, -0.6428,  -0.265, 0.0009189, -0.0009298, -0.2695, -0.6472, -0.9125, -0.9097, -0.6405, -0.2628, 0.001827, -0.001871, -0.2717, -0.6494, -0.9134, -0.9088, -0.6383, -0.2606, 0.002724, -0.002822,  -0.274, -0.6517, -0.9143, -0.9078]])

In [21]:
potencia_iterada(N(Q), x, 100, devolver_ultimo=False)[:, -5:]  # No hay una convergencia

Matrix([
[ 1.37557034955669,    1.09950394431198,  0.87958650631057, 0.703627646520887, 0.599249037699484],
[0.798807773712383,  -0.098973906538541,  14.1485870868619,  1.52866460582954, 0.880853879297849],
[0.389731890617887, -0.0427717673838562, -1.15251841222703,  24.7932563995652,  2.28101317939026]])

### Método QR

El método QR consiste en aplicar los siguientes pasos:

$$A^{(1)} = A$$

De ahí sacamos que 
$$A^{(1)}  = Q^{(1)}R^{(1)}$$

De ahí construimos:
$$A^{(2)}  = R^{(1)}Q^{(1)}$$

Y se cumple que $A^{(1)}$ y $A^{(2)}$ son semejantes, luego tienen los mismos 
autovalores.

Con ello se reitera el proceso, y se cumple que las matrices equivalentes 
construidas convergen a una matriz triangular superior. Los la diagonal de $A^{(k)}$ converge a los autovalores de $A$.

In [22]:
help(metodo_autovals_QR)

Help on function metodo_autovals_QR in module anmi.T3:

metodo_autovals_QR(A, n_iters=3, verbose=True)
    Aplica el método QR para el cálculo de autovalores de una matriz.
    
    Args:
        A (matriz): Matriz para el metodo
        n_iters (int, optional): Número de iteraciones. Defaults to 3.
        verbose (bool, optional): Imprime información intermedia.
    
    Returns:
        dict: "A": lista de valores de A = R*Q en cada iteración, "R" Y "Q": matrices Q y R del método.



In [23]:
dict_QR = metodo_autovals_QR(A, n_iters=10)

Aplicamos QR con Gram Schmidt
Q es 
[[2*sqrt(5)/5 3*sqrt(70)/70 sqrt(14)/14]
 [-sqrt(5)/5 3*sqrt(70)/35 sqrt(14)/7]
 [0 -sqrt(70)/14 3*sqrt(14)/14]]
R es 
[[sqrt(5) -4*sqrt(5)/5 sqrt(5)/5]
 [0 sqrt(70)/5 -8*sqrt(70)/35]
 [0 0 2*sqrt(14)/7]]
D es 
[[sqrt(5) 0 0]
 [0 sqrt(70)/5 0]
 [0 0 2*sqrt(14)/7]]
Aplicamos QR con Gram Schmidt
Q es 
[[sqrt(210)/15 sqrt(805)/115 sqrt(690)/345]
 [-sqrt(15)/15 7*sqrt(230)/115 2*sqrt(2415)/345]
 [0 -sqrt(46)/23 sqrt(483)/23]]
R es 
[[sqrt(210)/5 -12*sqrt(15)/35 2*sqrt(3)/21]
 [0 sqrt(230)/7 -20*sqrt(46)/161]
 [0 0 2*sqrt(483)/69]]
D es 
[[sqrt(210)/5 0 0]
 [0 sqrt(230)/7 0]
 [0 0 2*sqrt(483)/69]]
Aplicamos QR con Gram Schmidt
Q es 
[[11*sqrt(4494)/749 233*sqrt(1781871)/1781871 2*sqrt(16653)/16653]
 [-sqrt(17227)/749 2563*sqrt(27322022)/13661011 22*sqrt(255346)/127673]
 [0 -2*sqrt(5854719)/54717 233*sqrt(54717)/54717]]
R es 
[[sqrt(4494)/21 -124*sqrt(17227)/17227 2*sqrt(14766)/7383]
 [0 sqrt(27322022)/2461 -192*sqrt(5854719)/1951573]
 [0 0 2*sqrt(54717)/7

In [24]:
N(dict_QR['A'][-2], 3)

Matrix([
[   3.41,  -0.0162,        0],
[-0.0162,      2.0, -3.17e-5],
[      0, -3.17e-5,    0.586]])

In [25]:
N(dict_QR['A'][-1], 3)

Matrix([
[    3.41, -0.00952,        0],
[-0.00952,      2.0, -9.29e-6],
[       0, -9.29e-6,    0.586]])

In [26]:
N(2- sqrt(2), 3), 2, N(2 + sqrt(2), 3), 

(0.586, 2, 3.41)

In [27]:
A = mat([[1, 1, 1], [0, 0, 1], [0, 1, 1]])
dict_QR = metodo_autovals_QR(A, n_iters=30, verbose=False)

In [28]:
N(dict_QR['A'][-15], 3)

Matrix([
[1.0,    1.38,  -0.325],
[  0,    1.62, 7.43e-7],
[  0, 7.43e-7,  -0.618]])

In [29]:
N(dict_QR['A'][-1], 3)

Matrix([
[1.0,     1.38,   -0.325],
[  0,     1.62, 1.05e-12],
[  0, 1.05e-12,   -0.618]])

In [30]:
[N(i, 3) for i in list(A.eigenvals().keys())]

[1.00, -0.618, 1.62]

## Ejercicios

### Ejercicio 27

Determinar las primera iteraciones generadas por el método de la potencia con normalización con norma infinito cuando se aplica a la matriz con vector inicial.

In [31]:
A = Matrix([[0, -1, 1], [0, 1, -1], [-1, -1, 2]])
A

Matrix([
[ 0, -1,  1],
[ 0,  1, -1],
[-1, -1,  2]])

In [32]:
x0 = Matrix([1, -1, 2])

In [49]:
n_iters = 15
m_krylov_x, m_krylov_w = zeros(A.shape[0], n_iters), zeros(A.shape[0], n_iters)
m_krylov_x[:, 0] = x0
m_krylov_w[:, 0] = x0/max(x0)

for i in range(1, n_iters):
    kriv_i_x = A * m_krylov_w[:, i - 1]
    m_krylov_x[:, i] = kriv_i_x 
    m_krylov_w[:, i] = kriv_i_x / max(kriv_i_x) 


In [50]:
m_krylov_x

Matrix([
[ 1,  3/2,  7/4,  15/8,  31/16,  63/32,  127/64,  255/128,  511/256,  1023/512,  2047/1024,  4095/2048,  8191/4096,  16383/8192,  32767/16384],
[-1, -3/2, -7/4, -15/8, -31/16, -63/32, -127/64, -255/128, -511/256, -1023/512, -2047/1024, -4095/2048, -8191/4096, -16383/8192, -32767/16384],
[ 2,    2,    2,     2,      2,      2,       2,        2,        2,         2,          2,          2,          2,           2,            2]])

In [51]:
m_krylov_w

Matrix([
[ 1/2,  3/4,  7/8,  15/16,  31/32,  63/64,  127/128,  255/256,  511/512,  1023/1024,  2047/2048,  4095/4096,  8191/8192,  16383/16384,  32767/32768],
[-1/2, -3/4, -7/8, -15/16, -31/32, -63/64, -127/128, -255/256, -511/512, -1023/1024, -2047/2048, -4095/4096, -8191/8192, -16383/16384, -32767/32768],
[   1,    1,    1,      1,      1,      1,        1,        1,        1,          1,          1,          1,          1,            1,            1]])

In [62]:
[N(i) for i in (np.array(m_krylov_x[0, 1:]) / np.array(m_krylov_w[0, :-1])).tolist()[0]]

[3.00000000000000,
 2.33333333333333,
 2.14285714285714,
 2.06666666666667,
 2.03225806451613,
 2.01587301587302,
 2.00787401574803,
 2.00392156862745,
 2.00195694716243,
 2.00097751710655,
 2.00048851978505,
 2.00024420024420,
 2.00012208521548,
 2.00006103888177]

In [63]:
# Probamos sin normalizar

n_iters = 15
m_krylov_x  = zeros(A.shape[0], n_iters)
m_krylov_x[:, 0] = x0

for i in range(1, n_iters):
    kriv_i_x = A * m_krylov_x[:, i - 1]
    m_krylov_x[:, i] = kriv_i_x 

m_krylov_x 

Matrix([
[ 1,  3,  7,  15,  31,  63,  127,  255,  511,  1023,  2047,  4095,  8191,  16383,  32767],
[-1, -3, -7, -15, -31, -63, -127, -255, -511, -1023, -2047, -4095, -8191, -16383, -32767],
[ 2,  4,  8,  16,  32,  64,  128,  256,  512,  1024,  2048,  4096,  8192,  16384,  32768]])

In [64]:
[N(i) for i in (np.array(m_krylov_x[0, 1:]) / np.array(m_krylov_x[0, :-1])).tolist()[0]]

[3.00000000000000,
 2.33333333333333,
 2.14285714285714,
 2.06666666666667,
 2.03225806451613,
 2.01587301587302,
 2.00787401574803,
 2.00392156862745,
 2.00195694716243,
 2.00097751710655,
 2.00048851978505,
 2.00024420024420,
 2.00012208521548,
 2.00006103888177]

### Ejercicio 28

Realizar una iteración con el método QR para el cálculo de autovalores de la matriz $A$

In [65]:
A = Matrix([[1, 1, 0], [2, 1, 0], [2, 0, 1]])

In [66]:
metodo_autovals_QR(A, n_iters=1, verbose=True)

Aplicamos QR con Gram Schmidt
Q es 
[[1/3 2/3 2/3]
 [2/3 1/3 -2/3]
 [2/3 -2/3 1/3]]
R es 
[[3 1 2/3]
 [0 1 -2/3]
 [0 0 1/3]]
D es 
[[3 0 0]
 [0 1 0]
 [0 0 1/3]]


{'A': [Matrix([
  [19/9, 17/9, 14/9],
  [ 2/9,  7/9, -8/9],
  [ 2/9, -2/9,  1/9]])], 'Q': [Matrix([
  [1/3,  2/3,  2/3],
  [2/3,  1/3, -2/3],
  [2/3, -2/3,  1/3]])], 'R': [Matrix([
  [3, 1,  2/3],
  [0, 1, -2/3],
  [0, 0,  1/3]])]}

In [71]:
dict_QR = metodo_autovals_QR(A, n_iters=15, verbose=False)
N(dict_QR['A'][-1])

AVISO!!! A != QR
AVISO!!! A != QR
AVISO!!! A != QR
AVISO!!! A != QR
AVISO!!! A != QR
AVISO!!! A != QR
AVISO!!! A != QR


Matrix([
[    2.41421356236752,     1.15470542276318,   1.80738910771084],
[3.03592766377775e-12,    0.999998290811633, -0.632459353868169],
[6.78849140462537e-12, -3.82183366174131e-6,  -0.41421185317915]])

In [73]:
N(dict_QR['A'][-2])

Matrix([
[      2.4142135624056,    1.15468874635682,  -1.80739976177357],
[-1.76943180759392e-11,    1.00000412626917,   0.63244630524895],
[ 3.95663907028138e-11, -9.2267798802958e-6, -0.414217688674769]])

In [80]:
np.sort(solve(det(A - eye(3) * Symbol('lambda')), Symbol('lambda')))[::-1]

array([1 + sqrt(2), 1, 1 - sqrt(2)], dtype=object)

### Ejercicio 29
Sea $A$ con $\theta$ dado. Determinar el n-ésimo términa de la sucesión de Krylov asociada a $A$ y al vector $x$. Por medio de la sucesión de Krylov determia el polinomio característico de $A$.

In [89]:
t = Symbol('theta')
A = Matrix([[1, 0, 0], [0, cos(t), -sin(t)], [0, sin(t), cos(t)]])
x0 = Matrix([1, cos(t), -sin(t)])

In [95]:
ss = simplify(matriz_krylov(A, x0, n_iters=8))
ss

Matrix([
[          1, 1,          1,            1,            1,                                     1,                                                    1,                                                                                                1],
[ cos(theta), 1, cos(theta), cos(2*theta), cos(3*theta), 8*sin(theta)**4 - 8*sin(theta)**2 + 1, (16*sin(theta)**4 - 12*sin(theta)**2 + 1)*cos(theta), -sin(theta)**8 + 14*sin(theta)**6*cos(theta)**2 - 14*sin(theta)**2*cos(theta)**6 + cos(theta)**8],
[-sin(theta), 0, sin(theta), sin(2*theta), sin(3*theta),                          sin(4*theta), (16*sin(theta)**4 - 20*sin(theta)**2 + 5)*sin(theta),                                2*(16*sin(theta)**4 - 16*sin(theta)**2 + 3)*sin(theta)*cos(theta)]])

Así a priori tiene pintas de que $A^nx = (1, \cos(n-1)\theta, sin(n-1)\theta)^t$. 

Como $n=3$ solo necesitamos los primeros 4 términos de la sucesión de krylov para resolver el sistema y crear el polinomio característico.

In [96]:
M = ss[:3, :3]
am = Matrix([Symbol('a0'), Symbol('a1'), Symbol('a2')])
rhs = ss[:3, 3]

In [105]:
sol = simplify(M.inv() * rhs)
sol

Matrix([
[                                                                          1],
[                               (cos(theta) - cos(2*theta))/(cos(theta) - 1)],
[(-sin(theta) - sin(2*theta) + sin(3*theta))/(2*(cos(theta) - 1)*sin(theta))]])

In [114]:
l = Symbol('lambda')
simplify(sol[0] + sol[1] * l + sol[2] * l ** 2 - l ** 3)

(-lambda**2*(sin(theta) + sin(2*theta) - sin(3*theta)) + 2*lambda*(cos(theta) - cos(2*theta))*sin(theta) + (2 - 2*lambda**3)*(cos(theta) - 1)*sin(theta))/(2*(cos(theta) - 1)*sin(theta))